**Цель:**

Классификация последовательностей - это проблема прогнозирующего моделирования,
когда у вас есть некоторая последовательность входных данных в пространстве или
времени, и задача состоит в том, чтобы предсказать категорию для последовательности.
Проблема усложняется тем, что последовательности могут различаться по длине,
состоять из очень большого словарного запаса входных символов и могут потребовать от
модели изучения долгосрочного контекста или зависимостей между символами во входной
последовательности.
В данной лабораторной работе также будет использоваться датасет IMDb, однако
обучение будет проводиться с помощью рекуррентной нейронной сети.

**Задачи:**

Ознакомиться с рекуррентными нейронными сетями

Изучить способы классификации текста

Ознакомиться с ансамблированием сетей

Построить ансамбль сетей, который позволит получать точность не менее 97%

In [1]:
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
import numpy as np
from tkinter import *
from tkinter import ttk
import tkinter as tk
from tensorflow.keras.layers import Conv1D
from keras.layers import MaxPooling1D
from tensorflow.keras import layers

In [2]:
from keras.datasets import imdb  # Загружаем датесет IMDb,
(training_data, training_targets), (testing_data,
testing_targets) = imdb.load_data(num_words=10000)
data = np.concatenate((training_data, testing_data), axis=0)

targets = np.concatenate((training_targets, testing_targets),axis=0)  # объединяем, потом разделим по другому


top_words = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)
max_review_length = 500
X_train = sequence.pad_sequences(X_train,
maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test,
maxlen=max_review_length)


embedding_vecor_length = 32  # На вход получает номера слов, а на выходе выдаёт их векторные представления
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length,
input_length=max_review_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam',
metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, validation_data=(X_test, y_test),
epochs=3, batch_size=64)
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 32)           160000    
                                                                 
 lstm (LSTM)                 (None, 100)               53200     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/3
391/391 [==============================] - 308s 775ms/step - loss: 0.4810 - accuracy: 0.7659 - val_loss: 0.3577 - val_accuracy: 0.8504
Epoch 2/3
391/391 [==============================] - 324s 830ms/step - loss: 0.3403 - accuracy: 0.8534 - val_loss: 0.3354 - val_accuracy: 0.8612
Epoch 3/3
391/391 [

In [3]:
from keras.datasets import imdb
(training_data, training_targets), (testing_data,
testing_targets) = imdb.load_data(num_words=10000)
data = np.concatenate((training_data, testing_data), axis=0)

targets = np.concatenate((training_targets, testing_targets),
axis=0)

top_words = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)
max_review_length = 500
X_train = sequence.pad_sequences(X_train,
maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test,
maxlen=max_review_length)

embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam',
metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, validation_data=(X_test, y_test),
epochs=3, batch_size=64)
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 500, 32)           160000    
                                                                 
 conv1d (Conv1D)             (None, 500, 32)           3104      
                                                                 
 max_pooling1d (MaxPooling1D  (None, 250, 32)          0         
 )                                                               
                                                                 
 lstm_1 (LSTM)               (None, 100)               53200     
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 216,405
Trainable params: 216,405
Non-trainable params: 0
________________________________________________

In [13]:
from keras.datasets import imdb # меняем отношение тренировочных и тестовых данных
from keras.layers import Dropout
top_words = 5000
(training_data, training_targets), (testing_data, testing_targets) = imdb.load_data(num_words=top_words)
data = np.concatenate((training_data, testing_data), axis=0)
targets = np.concatenate((training_targets, testing_targets), axis=0)

X_test = data[:250]
y_test = targets[:250]
X_train = data[1:]
y_train = targets[1:]

max_review_length = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)

embedding_vecor_length = 32 # длина вектора, в котором будут представляться слова

model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length)) # плотный вектор
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.3))
model.add(LSTM(100)) # рекуррентный слой
model.add(Dropout(0.2))
model.add(Dense(128, activation="relu"))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3, batch_size=64)
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Epoch 1/3
782/782 [==============================] - 265s 336ms/step - loss: 0.3427 - accuracy: 0.8435 - val_loss: 0.1662 - val_accuracy: 0.9360
Epoch 2/3
782/782 [==============================] - 265s 339ms/step - loss: 0.2319 - accuracy: 0.9091 - val_loss: 0.1461 - val_accuracy: 0.9440
Epoch 3/3
782/782 [==============================] - 242s 310ms/step - loss: 0.2035 - accuracy: 0.9214 - val_loss: 0.1133 - val_accuracy: 0.9600
Accuracy: 96.00%


In [14]:
index = imdb.get_word_index()
def predict(txt:str):
    txt = txt.lower()
    txt1 = ""
    for i in txt:
        if('a'<=i<='z' or i==' '):
            txt1+=i
    txt1=txt1.split()
    tokens=np.array([min(index.get(i, 5000),5000)+3 for i in txt1])
    vector = sequence.pad_sequences([tokens], maxlen=max_review_length)
    p=model.predict(vector)[0][0]
    return "POSITIVE "+str(p) if p>0.5 else "NEGATIVE "+str(1-p)

predict("very very bad film")

'NEGATIVE 0.8860284686088562'